In [ ]:
!pip install requests PyMuPDF

In [4]:
!pip install ftfy

# Ollama Code aus Git

In [1]:
import requests
import json
import re
import ftfy

class OllamaApi:

    HOST = "https://f2ki-h100-1.f2.htw-berlin.de"
    PORT = 11435

    TIMEOUT = 120
    STREAM_RESPONSE = False # Only for debug purposes. !! Result will be null !!

    THINKING = False

    FALSE_RETURN = {"result": None, "time": 0, "token": 0, "info": {}}

    DEFAULT_OPTIONS = {
        "num_ctx": 2048,        # Default: 2048
        "repeat_last_n": 64,    # Default: 64, 0 = disabled, -1 = num_ctx
        "repeat_penalty": 1.1,  # Default: 1.1
        "temperature": 0.8,     # Default: 0.8
        "seed": 0,              # Default: 0
        "stop": [],             # No default
        "num_predict": -1,      # Default: -1, infinite generation
        "top_k": 40,            # Default: 40
        "top_p": 0.9,           # Default: 0.9
        "min_p": 0.0            # Default: 0.0
    }

    @staticmethod
    def fix_invalid_escapes(s):
        if not isinstance(s, str):
            return s

        try:
            s_re = ftfy.fix_text(s)
            if s_re != s:
                s = s_re
        except Exception as e:
            print(f"Encoding failed for text: {e}")

        return s

    @classmethod
    def models(cls):
        url = f"{cls.HOST}:{cls.PORT}/api/tags"
        headers = {
            "accept": "application/json",
        }
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Request failed with status {response.status_code}: {response.text}")
            return False
        else:
            try:
                json_data = response.json()
                return json_data.get("models", [])

            except json.JSONDecodeError as e:
                print(f"Failed to parse JSON: {e}")
                return False

    @classmethod
    def pull_model(cls, name:str, tag:str):
        url = f"{cls.HOST}:{cls.PORT}/api/pull"
        headers = {
            "Content-Type": "application/json",
            "accept": "application/json"
        }
        payload = {
            "model": f"{name}:{tag}"
        }
        try:
            response = requests.post(url, headers=headers, json=payload, stream=True, timeout=cls.TIMEOUT)
            if response.status_code != 200:
                print(f"Pull request failed with status {response.status_code}: {response.text}")
                return False

            for line in response.iter_lines():
                if line:
                    try:
                        progress = json.loads(line)
                        if 'status' in progress:
                            print(f"Pulling model: {progress['status']}")
                        if progress['status'] == "success":
                            return True
                    except json.JSONDecodeError:
                        continue

            return True
        except Exception as e:
            print(f"Failed to pull model: {e}")
            return False

    @classmethod
    def completion(cls, prompt:str, model="phi4:latest", schema=None, options=None):
        payload = {
            "model": model,
            "prompt" : prompt,
            "options": {
                **cls.DEFAULT_OPTIONS,
                **options
            } if options is not None else cls.DEFAULT_OPTIONS,
        }
        if schema is not None:
            payload["format"] = schema

        return cls.api_request(payload, force_json=False if schema is None else True)

    @classmethod
    def chat(cls, chat, model="phi4:latest", schema=None, options=None):
        payload = {
            "model": model,
            "messages": chat,
            "options": {
                **cls.DEFAULT_OPTIONS,
                **options
            } if options is not None else cls.DEFAULT_OPTIONS,
        }
        if schema is not None:
            payload["format"] = schema

        return cls.api_request(payload, force_json=False if schema is None else True)

    @classmethod
    def api_request(cls, payload, force_json:bool):
        if "messages" in payload:
            # Chat Request
            url = f"{cls.HOST}:{cls.PORT}/api/chat"
        else:
            # Completion Request
            url = f"{cls.HOST}:{cls.PORT}/api/generate"

        headers = {
            "Content-Type": "application/json",
            "accept": "application/json"
        }

        payload = {
            **payload,
            "think": cls.THINKING,
            "stream": cls.STREAM_RESPONSE,
            "keep_alive": "5m"
        }

        try:
            response = requests.post(url, headers=headers, json=payload, stream=cls.STREAM_RESPONSE, timeout=cls.TIMEOUT)

            if cls.STREAM_RESPONSE:
                for line in response.iter_lines(decode_unicode=True):
                    try:
                        chunk = json.loads(line)

                        if 'message' in chunk and 'content' in chunk['message']:
                            content = chunk['message']['content']
                            print(content, end='', flush=True)

                        if 'done' in chunk and chunk['done']:
                            break  # Exit the loop if done is True
                    except json.JSONDecodeError as e:
                        print(f"ERROR: Failed to decode JSON during streaming: {e}")
                return {**cls.FALSE_RETURN, "info": {"error": 'Streaming mode does not retreive a value'}}
            else:
                return cls.secure_json_response(response) if force_json else cls.secure_text_response(response)

        except requests.exceptions.Timeout:
            print(f"ERROR: The request took to long. Adjust the timeout ({cls.TIMEOUT}) as needed")
            return {**cls.FALSE_RETURN, "info": {"error": f"Request timeout ({cls.TIMEOUT}) reached"}}
        except Exception as e:
            print(f"ERROR: Request exception: {e}")
            return {**cls.FALSE_RETURN, "info": {"error": f"Request exception: {e}"}}


    @classmethod
    def secure_json_response(cls, response):

        text_response = cls.secure_text_response(response)

        if text_response.get("result") is None:
            return text_response

        message = str(text_response.get("result"))

        markdown_response = False
        thinking_block = False

        if message.strip().startswith("<think>"):
            print('WARN: Model returned <think> reasoning block before JSON')
            message = re.sub(r"^\s*<think>.*?</think>\s*", "", message, flags=re.DOTALL).strip()
            thinking_block = True

        match = re.search(r'```json(.*?)```', message, re.DOTALL)
        if match:
            # Remove everything except the content in "```json" to "```"
            print('WARN: Model returned markdown instead of only JSON')
            message = match.group(1).strip()
            markdown_response = True

        try:
            # Try to parse To json
            result = json.loads(message)

            # Overwrite text result with json dict
            text_response["result"] = dict(result)
            text_response["info"] = {
                "thinking": thinking_block,
                "markdown": markdown_response
            }
            return text_response

        except json.JSONDecodeError as e:
            print(f"ERROR: Failed to decode JSON: {e}")
            return {**cls.FALSE_RETURN, "info": {"error": 'JSON decode error on the model\'s response'}}

        except Exception as e:
            print(f"ERROR: Failed to parse JSON: {e}")
            return {**cls.FALSE_RETURN, "info": {"error": str(e)}}

    @classmethod
    def secure_text_response(cls, response):

        try:
            parsed_json = response.json()

            if response.status_code != 200:
                err_msg = parsed_json.get('error', 'Unknown error')
                print(f"ERROR: Request failed with status {response.status_code}: {err_msg}")
                return {**cls.FALSE_RETURN, "info":{"error":err_msg}}

            if 'done' not in parsed_json or parsed_json.get('done') is False:
                print("ERROR: Response has returned but Model didn't complete the answer")
                return {**cls.FALSE_RETURN, "info": {"error": 'Incomplete answer'}}

            # LLM Chat return as string
            message = parsed_json.get('message').get('content') if "message" in parsed_json else parsed_json.get('response')

            microseconds_elapsed = parsed_json.get('total_duration')
            seconds_elapsed = round(microseconds_elapsed / 1000000000, 3)
            token_count = parsed_json.get('eval_count')

            return {
                "result": cls.fix_invalid_escapes(message),
                "time": float(seconds_elapsed),
                "token": int(token_count),
                "info": {}
            }

        except json.JSONDecodeError as e:
            print(f"ERROR: Failed to decode JSON: {e}")
            return {**cls.FALSE_RETURN, "info": {"error": 'JSON decode error on the ollama server\'s response'}}

        except Exception as e:
            print(f"ERROR: Failed to parse JSON: {e}")
            return {**cls.FALSE_RETURN, "info": {"error": str(e)}}


# Versprechen extrahieren

In [4]:
import os
import re
import fitz  # PyMuPDF

# Modell wählen
model_name = "llama3.1:70b" #Sehr leistungsfähig, gute Sprachverarbeitung auch auf Deutsch, Gute Wahl für präzise Informationsextraktion
print(f"-------\nUsing model: {model_name}")

# Ordnerpfad mit PDFs (anpassen)
pdf_folder_path = "./Berichte"

# Funktion zur Extraktion von Text aus einer PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        doc = fitz.open(pdf_path)
        for page in doc:
            text += page.get_text()
        doc.close()
    except Exception as e:
        print(f"Fehler beim Lesen von {pdf_path}: {e}")
    return text

# Funktion zum Chunken von Text basierend auf Satzgrenzen
def chunk_text_into_sentences(text, max_chunk_chars=3000):
    # Zerlege den Text an Satzzeichen + Leerzeichen (nach Punkt, Fragezeichen oder Ausrufezeichen)
    sentences = re.split(r'(?<=[.!?])\s+', text)
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 <= max_chunk_chars:
            current_chunk += " " + sentence
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

# Prompt-Vorlage
prompt_template = """
Extrahiere bitte alle Nachhaltigkeitsversprechen die ein konkretes Ziel verfolgen aus folgendem Text. Gib nur den relevanten Text zurück, keine Einleitung oder sonstigen Text:

{text}
"""

# PDFs iterieren
for filename in os.listdir(pdf_folder_path):
    if filename.endswith(".pdf"):
        full_path = os.path.join(pdf_folder_path, filename)
        print(f"\n---\nVerarbeite Datei: {filename}")

        # PDF-Text extrahieren
        pdf_text = extract_text_from_pdf(full_path)

        if not pdf_text.strip():
            print("PDF enthält keinen extrahierbaren Text.")
            continue

        # In Satzbasierte Chunks zerlegen
        chunks = chunk_text_into_sentences(pdf_text, max_chunk_chars=3000)
        print(f"PDF in {len(chunks)} Chunks aufgeteilt.")

        # Ergebnisse sammeln
        all_results = []

        for i, chunk in enumerate(chunks):
            print(f"\n→ Chunk {i+1}/{len(chunks)} wird analysiert...")
            prompt = prompt_template.format(text=chunk)

            try:
                result = OllamaApi.completion(prompt, model=model_name)
                extracted = result.get("result", "").strip()
                if extracted:
                    all_results.append(extracted)
            except Exception as e:
                print(f"Fehler bei der Modellabfrage (Chunk {i+1}): {e}")

        # Ergebnisse ausgeben
        print(f"\n### Extrahierte Nachhaltigkeitsversprechen aus {filename}:\n")
        for i, res in enumerate(all_results):
            print(f"--- Chunk {i+1} ---\n{res}\n")

-------
Using model: llama3.1:70b

---
Verarbeite Datei: 2023_Volkswagen_Group_Nachhaltigkeitsbericht.pdf
PDF in 179 Chunks aufgeteilt.

→ Chunk 1/179 wird analysiert...

→ Chunk 2/179 wird analysiert...

→ Chunk 3/179 wird analysiert...


KeyboardInterrupt: 

# extrahierte Versprechen automatisiert überprüfen

In [26]:
!pip install playwright
!playwright install


133.5 MiB [                    ] 0% 0.0s133.5 MiB [                    ] 0% 69.7s133.5 MiB [                    ] 0% 77.0s133.5 MiB [                    ] 0% 83.3s133.5 MiB [                    ] 0% 77.5s133.5 MiB [                    ] 0% 80.8s133.5 MiB [                    ] 0% 66.1s133.5 MiB [                    ] 0% 65.7s133.5 MiB [                    ] 0% 63.4s133.5 MiB [                    ] 0% 57.4s133.5 MiB [                    ] 0% 52.4s133.5 MiB [                    ] 0% 50.7s133.5 MiB [                    ] 0% 52.8s133.5 MiB [                    ] 0% 43.7s133.5 MiB [                    ] 0% 46.5s133.5 MiB [                    ] 0% 50.1s133.5 MiB [                    ] 0% 47.2s133.5 MiB [                    ] 0% 42.4s133.5 MiB [                    ] 0% 42.0s133.5 MiB [                    ] 1% 40.0s133.5 MiB [                    ] 1% 40.1s133.5 MiB [                    ] 1% 38.4s133.5 MiB [                    ] 1% 37.4s133.5 MiB [                    ] 1% 37.8s133.5 MiB [      

In [43]:
!pip install ddgs

In [ ]:
import requests
from bs4 import BeautifulSoup
from ddgs import DDGS
import time

# Initialisierung des Ollama-Clients (siehe Doku von htw-ollama-py)
OLLAMA_MODEL = "phi4:14b"

# Nachhaltigkeitsversprechen
claims = [
    "VW verspricht CO2-Neutralität bis 2050",
    "VW will den Anteil von E-Autos bis 2030 auf 70 Prozent erhöhen",
    "VW reduziert Emissionen in der Lieferkette um 30 Prozent bis 2030"
]

def suche_artikel(query, max_results=5):
    with DDGS() as ddgs:
        return list(ddgs.text(query, max_results=max_results))

def scrape_text(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        r = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(r.text, "html.parser")
        paragraphs = soup.find_all('p')
        return '\n'.join(p.get_text() for p in paragraphs)
    except Exception as e:
        print(f"Fehler beim Abrufen von {url}: {e}")
        return ""

def analysiere_mit_ollama(text, claim):
    company = "Volkswagen"
    prompt = f"""
        "{company}" hat folgendes Nachhaltigkeitsversprechen abgegeben:

        "{claim}"

        Hier ist ein Ausschnitt eines Artikels zur Überprüfung:

        {text[:2000]}

        Basierend auf diesem Artikel:
        - Wurde das Versprechen eingehalten? (ja / teilweise / nein)
        - falls eine Strategie zur Erreichung des Ziels vorgestellt wurde, ist dies realistisch/umsetzbar?
        - Gib eine kurze Begründung.
        - Gib das Ergebnis im folgenden JSON-Format zurück:

        {{
        "status": "ja / teilweise / nein",
        "reason": "Begründung in einem Satz"
    }}
"""

    res = OllamaApi.completion(prompt, model=OLLAMA_MODEL)
    print("results: ", res.get("result"))
    try:
        return res.get("result")
    except Exception as e:
        print(f"❌ JSON-Fehler bei Auswertung: {e}")
        return None

def bewerte_claim(claim, company):
    print(f"\n🔍 Suche nach Artikeln zum Thema: {claim}")
    #ergebnisse = suche_artikel(f"intitle:{company} AND intitle:{claim} after:2023" ) #site:spiegel.de OR site:zeit.de OR site:sueddeutsche.de")
    ergebnisse = suche_artikel(f"(({claim} OR '{company} Nachhaltigkeitsziele' OR '{company} Greenwashing') AND (site:nachhaltigkeit-wirtschaft.de)) after:2024-01-01") #Klammern um die site-Bedingungen, damit sie sich auf alle Suchbegriffe beziehen

    for eintrag in ergebnisse:
        url = eintrag.get("href", "")
        if not url:
            continue
        print(f"📄 Artikel gefunden: {url}")
        text = scrape_text(url)
        if not text:
            continue
        bewertung = analysiere_mit_ollama(text, claim)
        print(f"🤖 Bewertung durch Ollama:\n{bewertung}\n")
        time.sleep(3)  # Vermeidung von Rate Limits

# Hauptdurchlauf
for c in claims:
    company = "Volkswagen"
    bewerte_claim(c, company)



🔍 Suche nach Artikeln zum Thema: VW verspricht CO2-Neutralität bis 2050
📄 Artikel gefunden: https://nachhaltigkeit-wirtschaft.de/glossar/kohlenstoffkreislauf/
📄 Artikel gefunden: https://nachhaltigkeit-wirtschaft.de/glossar/smart-grid/
results:  Basierend auf dem Artikel wurden keine spezifischen Informationen über Volkswagen oder dessen Strategie zur Erreichung der CO2-Neutralität bis 2050 bereitgestellt. Der Artikel konzentriert sich stattdessen allgemein auf die Vorteile und Rolle von Smart Grids im Kontext nachhaltiger Energiesysteme.

Hier ist das Ergebnis im angeforderten JSON-Format:

```json
{
    "status": "nein",
    "reason": "Der Artikel enthält keine spezifischen Informationen über Volkswagen oder dessen Maßnahmen zur Erreichung der CO2-Neutralität."
}
```

**Zusätzliche Bemerkungen:**

1. **Verbindung zu Smart Grids**: Während Smart Grids erheblich zum Übergang zu nachhaltigerer Energie beitragen können, deckt der Artikel nicht die spezifischen Pläne von Volkswagen ab.

